In [ ]:
%matplotlib inline
# Do not display warnings to prettify the notebook...
import warnings
warnings.simplefilter("ignore")

# Introduction

In this tutorial, we'll see how `nilearn` allows us to easily perform machine learning analyses with neuroimaging data, specifically MRI and fMRI. It is meant for people who do not necessarily have a background in programming.

You may notice that the name `nilearn` is reminiscent of [`scikit-learn`](https://scikit-learn.org), a popular Python library for machine learning.

This is no accident!

Nilearn and scikit-learn were created by the same team, and nilearn is designed to bring machine **LEARN**ing to the NeuroImaging (**NI**) domain.

## Getting Started

The following steps will help you get started quickly with nilearn. We start by trying to import the package. If not found, it will be installed.

In [ ]:
# import nilearn newest version (make sure it is 0.8.0 or more)
try:
    import nilearn
except:
    !pip install nilearn
    import nilearn
nilearn.__version__

## Outline

During this tutorial we will go over two Jupyter notebooks:

- **Part 1: Introduction (current notebook)**

    - <a href="#hello-world">"Hello World" of Nilearn: loading and plotting an image</a>
    - <a href="#smoothing">Run some manipulations</a>
    - <a href="#connectome">Creating and viewing a connectome with Nilearn</a>

- **Part 2: Run a classification task with Nilearn**

    - <a href="https://github.com/ymzayek/nilearn-PFC-Workshop/blob/main/Notebooks/classification-example.ipynb">See notebook `classification-example.ipynb`</a>

<div class="alert alert-block alert-info">
<b>Tip:</b> There are lots of fantastic tutorials on the <a href="https://nilearn.github.io">Nilearn website</a>. Learning to use Nilearn almost certainly starts here!
</div>

<div class="alert alert-block alert-success">
<b>Remember:</b> Don't be afraid to ask questions! And keep in mind that nobody ever writes error-free code!
</div>

## Basic nilearn example: manipulating and looking at data

We are now ready to use nilearn. The "Hello World" of nilearn consists in loading a volume and visualizing it.

Nilearn provides some utilities to facilitate the download of classic datasets. These functions are part of the [`datasets`](https://nilearn.github.io/stable/modules/datasets.html) module of Nilearn and all start with `fetch_`.

For example, the function [`nilearn.datasets.fetch_atlas_harvard_oxford`](https://nilearn.github.io/stable/modules/generated/nilearn.datasets.fetch_atlas_harvard_oxford.html) downloads the HarvardOxford atlas.

Nilearn is also shipped with some minimal data, for example the MNI 152 template:

In [ ]:
# Let's use a Nifti file that is shipped with nilearn
from nilearn.datasets import MNI152_FILE_PATH

# Note that the variable MNI152_FILE_PATH is just a path to a Nifti file
print('Path to MNI152 template: %r' % MNI152_FILE_PATH)

As you can see `MNI152_FILE_PATH` is nothing more than a Python string pointing to a file within the nilearn folder. We didn't download anything new here, the template is shipped with the nilearn source code.

In order to visualize the image stored at this location, we are going to use the [`plot_img`](https://nilearn.github.io/stable/modules/generated/nilearn.plotting.plot_img.html) function from the [`plotting`](https://nilearn.github.io/stable/modules/plotting.html) module of Nilearn.

Note that all functions related to visualization are defined in this module.

Let's quickly plot this file:

In [ ]:
from nilearn import plotting
plotting.plot_img(MNI152_FILE_PATH)

<span id="smoothing"></span>

As you probably imagine, a lot is happening under the hood here. We just passed a string representing the path to our image and [`plot_img`](https://nilearn.github.io/stable/modules/generated/nilearn.plotting.plot_img.html) took care of everything like loading the image in memory, finding the best cuts to display and so on...

Alternatively, you can use functions starting with `load_` from the `datasets` module to load different versions of the template and to set a desired resolution. Using this method loads objects directly into memory.

In [ ]:
from nilearn.datasets import load_mni152_template

# Load the same template but in lower resolution of 3mm
mni_template_3mm = load_mni152_template(resolution=3)
mni_template_3mm

## Simple image manipulation: smoothing

Now that we know how to visualize an image stored on disk, let's go a little bit further and do some manipulation to this image before plotting it.

Nilearn has an [`image`](https://nilearn.github.io/stable/modules/image.html) module which contains all functions related to image manipulations (loading, saving, indexing, resampling, and much more...).

In this simple example, we are going to use an image-smoothing function from this module: [`nilearn.image.smooth_img`](https://nilearn.github.io/stable/modules/generated/nilearn.image.smooth_img.html)

First we import the [`image`](https://nilearn.github.io/stable/modules/image.html) module:

In [ ]:
from nilearn import image

In a Jupyter notebook, you can very easily query the documentation using the magic command "?" that you prepend to the name of the function: 

In [ ]:
?image.smooth_img

Looking at the docstring, we see that this function expects:

- an image (or list of images), which can be either a filename or an in-memory image
- and the smoothing value in mm

In [ ]:
smooth_anat_img = image.smooth_img(mni_template_3mm, fwhm=3)

This is an in-memory object. As opposed to the path representation, we can query the object attributes:

In [ ]:
# Get the affine of the image
smooth_anat_img.affine

In [ ]:
# The image data is a 3D Numpy array
# containing 338752 values
smooth_anat_img.dataobj.shape

<div class="alert alert-block alert-warning">
<b>Warning:</b> It is important to keep in mind the memory footprint of loaded images as opposed to string representations.
</div>

Note that we can pass it to any nilearn function like we would a string representation, for instance to look at it:

In [ ]:
plotting.plot_img(smooth_anat_img)

# Creating a connectome with [`nilearn`](https://nilearn.github.io)

This is the last part of this notebook in which we will use nilearn to create a connectome from real data.

## Why do we need a specialized tool to work with neuroimaging data?

First of all, let's briefly consider why we might want specialized tools for working with neuroimaging data.

When performing a machine learning analysis, our data often look something like this:

In [ ]:
import pandas as pd

# read_csv can read in just about any plain-text tabular data
data = pd.read_csv('./data/abide2.tsv', sep='\t')
data.head()

For our purposes, what's most interesting is the structure of this data set. That is, the data is structured in a tabular format, with pre-extracted features of interest. In the machine learning world, the rows are usually called *samples* while columns refer to *features*.

This makes it easier to consider issues such as: which features would we like to predict? Or, how should we handle cross-validation?

But if we're starting with neuroimaging data, how can we create this kind of structured representation?

### Neuroimaging data

Neuroimaging data does not have a tabular structure. Instead, it has both **spatial and temporal dependencies** between successive data points. That is, knowing _where_ and _when_ something was measured tells you information about the surrounding data points.

We also know that neuroimaging data contains a lot of noise that's not blood-oxygen-level dependent (BOLD), such as head motion. Since we don't think that these other noise sources are related to neuronal firing, we often need to consider how we can make sure that our analyses are not driven by these noise sources.

These are all considerations that most machine learning software libraries are not designed to deal with! Nilearn therefore plays a crucial role in bringing machine learning concepts to the neuroimaging domain.

### Let's get our hands dirty!

To get a sense of the problem, the quickest method is to just look at some data. You may have your own data locally that you'd like to work with. As we saw in the introduction notebook, the [`datasets`](https://nilearn.github.io/stable/modules/datasets.html) module of Nilearn also provides utilities to easily download several neuroimaging data sets and atlases.

<div class="alert alert-block alert-success">
These data sets (and atlases) are only accessible because research groups chose to make their collected data publicly available. We owe them a huge thank you for this!
</div>

The data set we'll use today was originally collected by [Rebecca Saxe](https://mcgovern.mit.edu/profile/rebecca-saxe/)'s group at MIT and hosted on [OpenNeuro](https://openneuro.org/datasets/ds000228/versions/1.1.0).

The nilearn team preprocessed the data set with [fMRIPrep](https://fmriprep.readthedocs.io) and downsampled it to a lower resolution, so it'd be easier to work with.

We can learn a lot about this data set directly [from the Nilearn documentation](https://nilearn.github.io/stable/modules/generated/nilearn.datasets.fetch_development_fmri.html).

For example, we can see that this data set contains over 150 children and adults watching a short Pixar film.

Let's download the first 30 participants:

In [ ]:
from nilearn import datasets

development_dataset = datasets.fetch_development_fmri(n_subjects=30)
development_dataset.keys()

Now, this `development_dataset` object has several attributes which provide access to the relevant information.

For example, `development_dataset.phenotypic` provides access to information about the participants, such as whether they were children or adults.

`development_dataset.func` holds the functional MRI (fMRI) data. If you look at it, you will see that it is a list of 30 strings, where each string is a path to a file holding the images for a specific subject:

In [ ]:
len(development_dataset.func)

Let's have a closer look at the fMRI data of the first subject. As we saw previously, we can use functions from the [`image`](https://nilearn.github.io/stable/modules/image.html) module, for example [`image.load_img`](https://nilearn.github.io/stable/modules/generated/nilearn.image.load_img.html):

In [ ]:
from nilearn import image

img = image.load_img(development_dataset.func[0])
img.shape

This means that, for the first subject, there are 168 volumes, each with a 3D structure of (50, 59, 50).

<span id="subsetting"></span>

## Getting into the data: subsetting and viewing

As we saw in the introduction notebook, Nilearn provides many methods for plotting this kind of data.

We already saw [`nilearn.plotting.plot_img`](https://nilearn.github.io/stable/modules/generated/nilearn.plotting.plot_img.html) to display a static view of our image. Here we can use [`nilearn.plotting.view_img`](https://nilearn.github.io/stable/modules/generated/nilearn.plotting.view_img.html) to launch at interactive viewer.

On the contrary to the MNI 152 template image of the introduction notebook which was a 3D image, each fMRI run here is a 4D time series (three spatial dimensions plus time). This means that we will also need to subset the data when we plot it, so that we can look at a single 3D image.

Nilearn provides (at least) two ways to do this:

- with [`nilearn.image.index_img`](https://nilearn.github.io/stable/modules/generated/nilearn.image.index_img.html),
which allows us to index a particular frame--or several frames--of a time series,
- and with [`nilearn.image.mean_img`](https://nilearn.github.io/stable/modules/generated/nilearn.image.mean_img.html),
which allows us to take the mean 3D image over time.

Putting these together, we can iteratively view the mean image of the first participant using:

In [ ]:
from nilearn import plotting

mean_image = image.mean_img(img) # mean_image is 3D
plotting.view_img(mean_image, threshold=None)

<span id="extracting"></span>

## Extracting signal from fMRI volumes

As you can see, this data is decidedly not tabular!

What we would like is to extract and transform meaningful features from this data, and store it in a format that we can easily work with. Importantly, we _could_ work with the full time series directly. But we often want to reduce the dimensionality of our data in a structured way. That is, we may only want to consider signal within certain learned or pre-defined regions of interest (ROIs), and when taking into account known sources of noise. To do this, we'll use nilearn's Masker objects.

**What are the masker objects ?**

First, let's think about what masking fMRI data is doing:

In [ ]:
from IPython.display import Image
Image(filename='../images/masking.jpg')

Essentially, we can imagine overlaying a 3D grid on an image. Then, our mask tells us which cubes or “voxels” (like 3D pixels) to sample from.

Since our Nifti images are 4D files, we can’t overlay a single grid – instead, we use a series of 3D grids (one for each volume in the 4D file), so we can get a measurement for each voxel at each timepoint.

Masker objects allow us to apply these masks!

To start, we need to define a mask (or masks) that we'd like to apply. This could correspond to one or many regions of interest.

Nilearn provides methods to define your own functional parcellation (using clustering algorithms such as _k-means_), and it also provides access to other atlases that have previously been defined by researchers.

<span id="roi"></span>

### Choosing regions of interest

In this tutorial, we will use the MSDL (multi-subject dictionary learning, `Varoquaux_2011`) atlas, which defines a set of _probabilistic_ ROIs across the brain. 

<div class="alert alert-block alert-info">
<b>Tip:</b> Nilearn ships with several atlases commonly used in the field, including the Schaefer atlas and the Harvard-Oxford atlas.
</div>

To download the MSDL atlas, we rely on the function [`nilearn.datasets.fetch_atlas_msdl`](https://nilearn.github.io/stable/modules/generated/nilearn.datasets.fetch_atlas_msdl.html):

In [ ]:
import numpy as np

msdl_atlas = datasets.fetch_atlas_msdl()
msdl_atlas.keys()

In [ ]:
msdl_coords = msdl_atlas.region_coords
n_regions = len(msdl_coords)

print(f'MSDL has {n_regions} ROIs, part of the following networks :\n{np.unique(msdl_atlas.networks)}.')

`msdl_atlas.maps` contains a string which is the path to the maps images. You can have a look at them as you would with any other image:

In [ ]:
# Notice the data is 4D
image.load_img(msdl_atlas.maps).shape

As you can see, MSDL is a probabilistic atlas. We can view it using the function [`nilearn.plotting.plot_prob_atlas`](https://nilearn.github.io/stable/modules/generated/nilearn.plotting.plot_prob_atlas.html):

In [ ]:
plotting.plot_prob_atlas(msdl_atlas.maps)

<span id="nifti-zoo"></span>

### A quick side-note on the NiftiMasker zoo

We would like to supply these ROIs to a `Masker` object. All Masker objects share the same basic structure and functionality,
but each is designed to work with a different kind of ROI. Masker objects are defined in the [`maskers`](https://nilearn.github.io/stable/modules/maskers.html) module of nilearn.

The canonical [`nilearn.maskers.NiftiMasker`](https://nilearn.github.io/stable/modules/generated/nilearn.maskers.NiftiMasker.html) works well if we want to apply a single mask to the data, like a single region of interest.

But what if we actually have several ROIs that we'd like to apply to the data all at once?

If these ROIs are non-overlapping, as in "hard" or deterministic parcellations, then we can use [`nilearn.maskers.NiftiLabelsMasker`](https://nilearn.github.io/stable/modules/generated/nilearn.maskers.NiftiLabelsMasker.html).

Because we are working with "soft" or probabilistic ROIs, we can instead supply these ROIs to [`nilearn.maskers.NiftiMapsMasker`](https://nilearn.github.io/stable/modules/generated/nilearn.maskers.NiftiMapsMasker.html).

For a full list of the available Masker objects, see [the Nilearn documentation](https://nilearn.github.io/stable/modules/maskers.html).

<span id="apply-masker"></span>

### Applying a Masker object

We can supply our MSDL atlas-defined ROIs to the `NiftiMapsMasker` object, along with resampling, filtering, and detrending parameters.

In [ ]:
from nilearn import maskers

masker = maskers.NiftiMapsMasker(
    msdl_atlas.maps,
    resampling_target="data",
    t_r=2, 
    detrend=True,
    low_pass=0.1,
    high_pass=0.01
).fit()

One thing you might notice from the above code is that immediately after defining the masker object, we call the `.fit` method on it. This method may look familiar if you've previously worked with scikit-learn estimators!

You'll note that we're not supplying any data to this `.fit` method; that's because we're fitting the Masker to the provided ROIs, rather than to our data.

<span id="dim"></span>

### Dimensions, dimensions

We can use this fitted masker to `transform` our data.

In [ ]:
roi_time_series = masker.transform(development_dataset.func[0])
roi_time_series.shape

If you'll remember, when we first looked at the data its original dimensions were (50, 59, 50, 168).
Now, it has a shape of (168, 39).
What happened?!

Rather than providing information on every voxel within our original 3D grid,
we're now only considering those voxels that fall in our 39 regions of interest provided by the MSDL atlas and aggregating across voxels within those ROIS.
This reduces each 3D volume from a dimensionality of (50, 59, 50) to just 39,
for our 39 provided ROIs.

You'll also see that the "dimensions flipped;"
that is, that we've transposed the matrix such that time is now the first rather than second dimension.
This follows the scikit-learn convention that rows in a data matrix are _samples_,
and columns in a data matrix are _features_.

In [ ]:
Image(filename="../images/samples-features.png")

```
---
The scikit-learn conventions for feature and target matrices.
From Jake VanderPlas's _Python Data Science Handbook_.
```

<div class="alert alert-block alert-success">
One of the nice things about working with nilearn is that it will impose this convention for you, so you don't accidentally flip your dimensions when using a scikit-learn model!
</div>

<span id="connectome"></span>

## Creating and viewing a connectome

The simplest and most commonly used kind of functional connectivity is pairwise correlation between ROIs.

We can estimate it using [`nilearn.connectome.ConnectivityMeasure`](https://nilearn.github.io/stable/modules/generated/nilearn.connectome.ConnectivityMeasure.html).

In [ ]:
from nilearn.connectome import ConnectivityMeasure

correlation_measure = ConnectivityMeasure(kind='correlation')
correlation_matrix = correlation_measure.fit_transform([roi_time_series])[0]

We can then plot this functional connectivity matrix using the function [`nilearn.plotting.plot_matrix`](https://nilearn.github.io/stable/modules/generated/nilearn.plotting.plot_matrix.html):

In [ ]:
import matplotlib.pyplot as plt

np.fill_diagonal(correlation_matrix, 0)
fig = plt.figure(figsize=(8, 8))
plotting.plot_matrix(
    correlation_matrix,
    labels=msdl_atlas.labels,
    vmax=0.8,
    vmin=-0.8,
    colorbar=True,
    figure=fig
)

Or view it as an embedded connectome using the function [`nilearn.plotting.view_connectome`](https://nilearn.github.io/stable/modules/generated/nilearn.plotting.view_connectome.html):

In [ ]:
plotting.view_connectome(
    correlation_matrix,
    edge_threshold=0.2,
    node_coords=msdl_atlas.region_coords
)

<span id="noise"></span>

## Accounting for noise sources

As we have already seen, maskers also allow us to perform other useful operations beyond just masking our data. One important processing step is correcting for measured signals of no interest (e.g., head motion).

Our `development_dataset` also includes several of these signals of no interest that were generated during fMRIPrep pre-processing. We can access these with the `confounds` attribute, using `development_dataset.confounds`.

Let's quickly check what these look like for our first participant:

In [ ]:
pd.read_table(development_dataset.confounds[0]).head()

We can see that there are several different kinds of noise sources included!
This is actually a subset of all possible fMRIPrep generated confounds that the Nilearn developers have pre-selected.
We could access the full list by passing the argument `reduce_confounds=False` to our original call downloading the `development_dataset`.
For most analyses, this list of confounds is reasonable, so we'll use these Nilearn provided defaults.
For your own analyses, make sure to check which confounds you're using!

Importantly, we can pass these confounds directly to our masker object:

In [ ]:
corrected_roi_time_series = masker.transform(
    development_dataset.func[0],
    confounds=development_dataset.confounds[0]
)

In [ ]:
corrected_correlation_matrix = correlation_measure.fit_transform(
    [corrected_roi_time_series])[0]
np.fill_diagonal(corrected_correlation_matrix, 0)
fig = plt.figure(figsize=(8, 8))
plotting.plot_matrix(
    corrected_correlation_matrix,
    labels=msdl_atlas.labels,
    vmax=0.8,
    vmin=-0.8,
    colorbar=True,
    figure=fig
)

As before, we can also view this functional connectivity matrix as a connectome:

In [ ]:
plotting.view_connectome(
    corrected_correlation_matrix,
    edge_threshold=0.2,
    node_coords=msdl_atlas.region_coords
)

In both the matrix and connectome forms, we can see a big difference when including the confounds!

<div class="alert alert-block alert-warning">
<b>Warning:</b> This is an important reminder to make sure that your data are cleaned of any possible sources of noise <em>before</em> running a machine learning analysis. Otherwise, you might be classifying participants on e.g. amount of head motion rather than a feature of interest!
</div>

## Conclusion

We reached the end of this first Notebook which helped us to get started with Nilearn.

We learned how to load an image, perform some basic manipulation, and visualize it. By doing so, we discovered the following modules of Nilearn:

- [`datasets`](https://nilearn.github.io/stable/modules/datasets.html): Contains all utilities to load data
- [`plotting`](https://nilearn.github.io/stable/modules/plotting.html): Contains all utilities to visualize images
- [`image`](https://nilearn.github.io/stable/modules/image.html): Contains all utilities to manipulate images

We also learned how to build a connectome from fMRI data. More specifically, we learned how to extract signals from fMRI volumes by defining regions of interests and using a masker object. We also learned how to use [`ConnectivityMeasure`](https://nilearn.github.io/stable/modules/generated/nilearn.connectome.ConnectivityMeasure.html) objects to compute the strengths of connections. In addition to the [`datasets`](https://nilearn.github.io/stable/modules/datasets.html), [`image`](https://nilearn.github.io/stable/modules/image.html), and [`plotting`](https://nilearn.github.io/stable/modules/plotting.html) modules from the first notebook, we discovered here the following modules:

- [`maskers`](https://nilearn.github.io/stable/modules/maskers.html): Contains all maskers
- [`connectome`](https://nilearn.github.io/stable/modules/connectome.html): Contains tools for computing functional connectivity matrices

There exists many more modules in Nilearn. We will use some of them in the next parts.

<div class="alert alert-block alert-info">
<b>Tip:</b>Note that you can always refer to the <a href="https://nilearn.github.io/stable/modules/reference.html">API documentation</a> of Nilearn. This is a tool that all users of Nilearn refer to, so do not hesitate to use it!
</div>